In [2]:
import os
import shutil
import pandas as pd
import fiftyone as fo
import fiftyone.zoo as foz
from PIL import Image
import numpy as np

## Import images from Open Images Dataset V7 

In [1]:
# Clase de interés
class_of_interest = "Bird"
num_img = 1000 # Número máximo de muestras a descargar
split = "train" # Puedes cambiar a "train" o "test" según tus necesidades

In [ ]:
dataset = foz.download_zoo_dataset(
    "open-images-v7",
    split=split,
    label_types=["segmentations"],
    classes=[class_of_interest],
    max_samples=num_img,
    shuffle=True
)

#### Download all separated masks

In [ ]:
# Directorios base
base_dir = "/Users/kevinguzmanhuamani/fiftyone/open-images-v7/train"
data_dir = os.path.join(base_dir, "data")  # Directorio donde están las imágenes descargadas
masks_dir = os.path.join(base_dir, "labels/masks")  # Directorio de máscaras organizadas por subcarpetas (0-F)
metadata_dir = os.path.join(base_dir, "metadata")  # Directorio de metadatos
output_dir = "/Users/kevinguzmanhuamani/Desktop/dat"  # Directorio principal de salida

# Crear subdirectorios para imágenes y máscaras
images_output_dir = os.path.join(output_dir, "images")
masks_output_dir = os.path.join(output_dir, "masks")
os.makedirs(images_output_dir, exist_ok=True)
os.makedirs(masks_output_dir, exist_ok=True)

# Leer el archivo de clases (classes.csv) para obtener el identificador de la clase
classes_csv = os.path.join(metadata_dir, "classes.csv")
classes = pd.read_csv(classes_csv, header=None, names=["LabelName", "ClassName"])
class_label = classes[classes["ClassName"] == class_of_interest]["LabelName"].values[0]  # Identificador único
class_label = class_label.replace("/m/", "m")  # Formatear identificador para coincidir con los nombres de las máscaras

# Obtener los nombres base de las imágenes en "data" (sin extensión .jpg)
image_names = [os.path.splitext(f)[0] for f in os.listdir(data_dir) if f.endswith(".jpg")]

# Copiar imágenes al directorio de salida
for image_file in os.listdir(data_dir):
    if image_file.endswith(".jpg"):
        source_path = os.path.join(data_dir, image_file)
        destination_path = os.path.join(images_output_dir, image_file)
        shutil.copy(source_path, destination_path)

# Iterar sobre las subcarpetas de "masks" (0-F) y copiar máscaras correspondientes
for subfolder in os.listdir(masks_dir):
    subfolder_path = os.path.join(masks_dir, subfolder)
    
    # Asegurarse de que sea una carpeta
    if not os.path.isdir(subfolder_path):
        continue
    
    # Buscar máscaras dentro de esta subcarpeta
    for mask_file in os.listdir(subfolder_path):
        # Verificar si el nombre sigue el formato <ImageID>_<ClassLabel>_
        parts = mask_file.split("_")
        if len(parts) >= 2:  # Asegurar que haya al menos <ImageID>_<ClassLabel>
            mask_base_name, mask_class = parts[0], parts[1]
            
            # Comprobar si cumple la combinación <ImageID>_<ClassLabel>
            if mask_base_name in image_names and mask_class == class_label:
                source_path = os.path.join(subfolder_path, mask_file)
                destination_path = os.path.join(masks_output_dir, mask_file)
                
                # Copiar la máscara al directorio de máscaras
                shutil.copy(source_path, destination_path)

print(f"Imágenes copiadas al directorio: {images_output_dir}")
print(f"Máscaras copiadas al directorio: {masks_output_dir}")

#### Download only one separated masks

In [ ]:
# Directorios base
base_dir = "/Users/kevinguzmanhuamani/fiftyone/open-images-v7/train"
data_dir = os.path.join(base_dir, "data")  # Directorio donde están las imágenes descargadas
masks_dir = os.path.join(base_dir, "labels/masks")  # Directorio de máscaras organizadas por subcarpetas (0-F)
metadata_dir = os.path.join(base_dir, "metadata")  # Directorio de metadatos
output_dir = "/Users/kevinguzmanhuamani/Desktop/dat"  # Directorio principal de salida

# Crear subdirectorios para imágenes y máscaras
images_output_dir = os.path.join(output_dir, "images")
masks_output_dir = os.path.join(output_dir, "masks")
os.makedirs(images_output_dir, exist_ok=True)
os.makedirs(masks_output_dir, exist_ok=True)

# Leer el archivo de clases (classes.csv) para obtener el identificador de la clase
classes_csv = os.path.join(metadata_dir, "classes.csv")
classes = pd.read_csv(classes_csv, header=None, names=["LabelName", "ClassName"])
class_label = classes[classes["ClassName"] == class_of_interest]["LabelName"].values[0]  # Identificador único
class_label = class_label.replace("/m/", "m")  # Formatear identificador para coincidir con los nombres de las máscaras

# Obtener los nombres base de las imágenes en "data" (sin extensión .jpg)
image_names = [os.path.splitext(f)[0] for f in os.listdir(data_dir) if f.endswith(".jpg")]

# Copiar imágenes al directorio de salida
for image_file in os.listdir(data_dir):
    if image_file.endswith(".jpg"):
        source_path = os.path.join(data_dir, image_file)
        destination_path = os.path.join(images_output_dir, image_file)
        shutil.copy(source_path, destination_path)

# Usar un conjunto para registrar las imágenes que ya tienen una máscara asociada
processed_images = set()

# Iterar sobre las subcarpetas de "masks" (0-F) y copiar una sola máscara por imagen
for subfolder in os.listdir(masks_dir):
    subfolder_path = os.path.join(masks_dir, subfolder)
    
    # Asegurarse de que sea una carpeta
    if not os.path.isdir(subfolder_path):
        continue
    
    # Buscar máscaras dentro de esta subcarpeta
    for mask_file in os.listdir(subfolder_path):
        # Verificar si el nombre sigue el formato <ImageID>_<ClassLabel>_
        parts = mask_file.split("_")
        if len(parts) >= 2:  # Asegurar que haya al menos <ImageID>_<ClassLabel>
            mask_base_name, mask_class = parts[0], parts[1]
            
            # Comprobar si cumple la combinación <ImageID>_<ClassLabel>
            if mask_base_name in image_names and mask_class == class_label:
                # Verificar si ya procesamos una máscara para esta imagen
                if mask_base_name not in processed_images:
                    source_path = os.path.join(subfolder_path, mask_file)
                    destination_path = os.path.join(masks_output_dir, mask_file)
                    
                    # Copiar la máscara al directorio de máscaras
                    shutil.copy(source_path, destination_path)
                    
                    # Registrar la imagen como procesada
                    processed_images.add(mask_base_name)

# Mensajes finales
print(f"Imágenes copiadas al directorio: {images_output_dir}")
print(f"Máscaras copiadas al directorio: {masks_output_dir}") 

####  Combine all masks into one 

In [ ]:
import os
import shutil
import pandas as pd
from PIL import Image
import numpy as np

# Directorios base
base_dir = "/Users/kevinguzmanhuamani/fiftyone/open-images-v7/train"
data_dir = os.path.join(base_dir, "data")  # Directorio donde están las imágenes descargadas
masks_dir = os.path.join(base_dir, "labels/masks")  # Directorio de máscaras organizadas por subcarpetas (0-F)
metadata_dir = os.path.join(base_dir, "metadata")  # Directorio de metadatos
output_dir = "/Users/kevinguzmanhuamani/Desktop/dat"  # Directorio principal de salida

# Crear subdirectorios para imágenes y máscaras
images_output_dir = os.path.join(output_dir, "images")
combined_masks_output_dir = os.path.join(output_dir, "combined_masks")  # Directorio para máscaras combinadas
os.makedirs(images_output_dir, exist_ok=True)
os.makedirs(combined_masks_output_dir, exist_ok=True)

# Leer el archivo de clases (classes.csv) para obtener el identificador de la clase
classes_csv = os.path.join(metadata_dir, "classes.csv")
classes = pd.read_csv(classes_csv, header=None, names=["LabelName", "ClassName"])
class_label = classes[classes["ClassName"] == class_of_interest]["LabelName"].values[0]  # Identificador único
class_label = class_label.replace("/m/", "m")  # Formatear identificador para coincidir con los nombres de las máscaras

# Obtener los nombres base de las imágenes en "data" (sin extensión .jpg)
image_names = [os.path.splitext(f)[0] for f in os.listdir(data_dir) if f.endswith(".jpg")]

# Copiar imágenes al directorio de salida
for image_file in os.listdir(data_dir):
    if image_file.endswith(".jpg"):
        source_path = os.path.join(data_dir, image_file)
        destination_path = os.path.join(images_output_dir, image_file)
        shutil.copy(source_path, destination_path)

# Combinar máscaras por imagen
for image_name in image_names:
    combined_mask = None  # Inicializar la máscara combinada
    
    # Iterar sobre las subcarpetas de "masks" (0-F)
    for subfolder in os.listdir(masks_dir):
        subfolder_path = os.path.join(masks_dir, subfolder)
        
        # Asegurarse de que sea una carpeta
        if not os.path.isdir(subfolder_path):
            continue
        
        # Buscar máscaras dentro de esta subcarpeta
        for mask_file in os.listdir(subfolder_path):
            # Verificar si el nombre sigue el formato <ImageID>_<ClassLabel>_
            parts = mask_file.split("_")
            if len(parts) >= 2:  # Asegurar que haya al menos <ImageID>_<ClassLabel>
                mask_base_name, mask_class = parts[0], parts[1]
                
                # Comprobar si cumple la combinación <ImageID>_<ClassLabel>
                if mask_base_name == image_name and mask_class == class_label:
                    source_path = os.path.join(subfolder_path, mask_file)
                    
                    # Cargar la máscara como una imagen binaria
                    mask = Image.open(source_path).convert("L")  # Convertir a escala de grises
                    
                    # Si no hay una máscara combinada, inicializarla
                    if combined_mask is None:
                        combined_mask = np.array(mask, dtype=np.uint8)
                    else:
                        # Combinar la máscara actual con la máscara combinada
                        combined_mask = np.maximum(combined_mask, np.array(mask, dtype=np.uint8))
    
    # Si se generó una máscara combinada, guardarla
    if combined_mask is not None:
        combined_mask_image = Image.fromarray(combined_mask)
        combined_mask_path = os.path.join(combined_masks_output_dir, f"{image_name}_combined.png")
        combined_mask_image.save(combined_mask_path)

# Mensajes finales
print(f"Imágenes copiadas al directorio: {images_output_dir}")
print(f"Máscaras combinadas guardadas en: {combined_masks_output_dir}")

## Preparation for Yolo schema

#### make polygons from masks

In [ ]:
import os
import cv2

# Rutas de entrada y salida
input_dir = '/Users/kevinguzmanhuamani/Desktop/dat/masks'
output_dir = '/Users/kevinguzmanhuamani/Desktop/dat/poligon'

# Crear el directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    
    # Cargar la máscara binaria y obtener sus contornos
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convertir los contornos en polígonos
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:  # Filtrar áreas pequeñas
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    # Generar el nombre del archivo de salida (antes del primer '_')
    base_name = j.split('_')[0]
    output_path = os.path.join(output_dir, base_name + '.txt')
    
    # Guardar los polígonos en el archivo de texto
    with open(output_path, 'w') as f:
        for polygon in polygons:
            for p_index, p in enumerate(polygon):
                if p_index == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_index == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

#### Preparate images and polygons for Yolo Schema (train/val = 70/30)

In [ ]:
import os
import shutil
import random

# Directorios
base_dir = '/Users/kevinguzmanhuamani/Desktop/dat'
images_dir = os.path.join(base_dir, 'images')
polygons_dir = os.path.join(base_dir, 'poligon')
yolo_dir = os.path.join(base_dir, 'YOLO')

# Crear estructura de directorios YOLO
images_train_dir = os.path.join(yolo_dir, 'images', 'train')
images_val_dir = os.path.join(yolo_dir, 'images', 'val')
labels_train_dir = os.path.join(yolo_dir, 'labels', 'train')
labels_val_dir = os.path.join(yolo_dir, 'labels', 'val')

for dir_path in [images_train_dir, images_val_dir, labels_train_dir, labels_val_dir]:
    os.makedirs(dir_path, exist_ok=True)

# Obtener los nombres base de las imágenes y etiquetas
files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
files_base = [os.path.splitext(f)[0] for f in files]  # Nombres sin extensiones

# Mezclar los nombres y dividir en 70/30
random.shuffle(files_base)
split_index = int(len(files_base) * 0.7)
train_files = files_base[:split_index]
val_files = files_base[split_index:]

# Mover archivos a las carpetas correspondientes
for base_name in train_files:
    # Copiar imágenes a train
    shutil.copy(os.path.join(images_dir, base_name + '.jpg'), os.path.join(images_train_dir, base_name + '.jpg'))
    # Copiar etiquetas a train
    shutil.copy(os.path.join(polygons_dir, base_name + '.txt'), os.path.join(labels_train_dir, base_name + '.txt'))

for base_name in val_files:
    # Copiar imágenes a val
    shutil.copy(os.path.join(images_dir, base_name + '.jpg'), os.path.join(images_val_dir, base_name + '.jpg'))
    # Copiar etiquetas a val
    shutil.copy(os.path.join(polygons_dir, base_name + '.txt'), os.path.join(labels_val_dir, base_name + '.txt'))

print("Archivos distribuidos correctamente en las carpetas YOLO.")